In [1]:
'''
데이터 셋 전처리

모델 선택 
CNN, LSTM, CNN-LSTM : "cnn"
BERT : "bert"
'''

csv = "cnn"


In [2]:
pip install utils

Note: you may need to restart the kernel to use updated packages.


In [3]:
pip install sklearn

Note: you may need to restart the kernel to use updated packages.


In [4]:
pip install scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [5]:
import os
import numpy as np
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_recall_fscore_support

In [6]:
#user_sleep_2020.csv

# 디렉토리 경로 설정
sleep_directory_path = "./2020/user_sleep_2020.csv"


# sleep_data 받아오기

sleep_data = pd.read_csv( sleep_directory_path, 
                         encoding="utf-8", 
                         usecols=["userId","date", "wakeupcount", "sleep_score",
                                  'startDt','endDt',
                                  'lightsleepduration', 'deepsleepduration','remsleepduration'
                                 ])

#sleepTime_col 추가
sleep_data["sleepTime"] = sleep_data['endDt'] - sleep_data['startDt']

# 'endDt','startDt' column 제거
sleep_data.drop(['endDt','startDt'], axis=1, inplace=True)

#sleepduration weight
sleep_data['lightsleepduration'] = round(sleep_data['lightsleepduration'] * 0.7, 1).astype(int)
sleep_data['deepsleepduration'] = round(sleep_data['deepsleepduration'] * 0.3, 1).astype(int)

# convert('userId', object, int)
sleep_data['userId'] = sleep_data['userId'].str.replace('user', '').astype(int)

# merge requirement
sleep_data['date'] = pd.to_datetime(sleep_data['date'])

sleep_data


,userId,date,lightsleepduration,deepsleepduration,wakeupcount,remsleepduration,sleep_score,sleepTime
0,24,2020-08-30,9660,2700,3,5280,86,32100
1,24,2020-08-31,6678,1908,1,4980,52,25800
2,24,2020-09-01,6132,1422,1,7440,61,22860
3,24,2020-09-02,5418,2232,1,2220,40,20160
4,24,2020-09-03,8148,2286,1,2040,55,25200
...,...,...,...,...,...,...,...,...
610,6,2020-10-05,8274,1368,1,5940,60,30780
611,6,2020-10-06,9240,2250,1,4680,79,27660
612,6,2020-10-07,5964,450,3,0,20,22440
613,6,2020-10-08,6468,1368,2,1080,20,19260


In [7]:
#user_info_2020.csv

user_info_path = "./2020/user_info_2020.csv"
user_info_data = pd.read_csv( user_info_path, 
                             encoding="utf-8", 
                             usecols=["userId","gender","age","height","weight"])

#bmi calculate
# user_info_data['bmi'] = user_info_data['weight'] / ((user_info_data['height']/100) ** 2)
user_info_data['bmi'] = round(user_info_data['weight'] / ((user_info_data['height']/100) ** 2), 0).astype(int)
user_info_data.drop(['height','weight'], axis=1, inplace=True)

#convert
user_info_data['userId'] = user_info_data['userId'].str.replace('user', '').astype(int)
user_info_data['gender'] = user_info_data['gender'].replace({'M': 1, 'F': 0})

#sorting
user_info_data.sort_values(by=['userId'], ascending=True, inplace=True)
user_info_data = user_info_data.reset_index(drop=True)

user_info_data

,userId,gender,age,bmi
0,1,1,27,27
1,2,0,25,29
2,3,0,32,22
3,4,1,28,24
4,5,1,31,24
5,6,0,22,26
6,7,1,30,28
7,8,1,32,26
8,9,1,20,28
9,10,1,25,23


In [8]:
#user01-30 life_data


# 디렉토리 경로 설정
directory = "./2020/user01-30"

# 모든 파일에 대해 작업 수행
results = []

for i in range(1,  31):
    if i < 13 or i > 20:
        subdirectory = os.path.join(directory, f"user{i:02d}")
        for filename in os.listdir(subdirectory):
            if not filename.startswith("."):
                filepath = os.path.join(subdirectory, filename)
                for a in os.listdir(filepath):
                    if not a.startswith("."):
                        if a.endswith(".csv"):
                            csv_file = os.path.join(filepath, a)
                            df = pd.read_csv(csv_file, encoding="utf-8")
                            df["userId"] = f"user{i:02d}"
                            results.append(df)

# 결과 데이터프레임 생성
life_data = pd.concat(results)
life_data = life_data[["userId", "ts", "actionOption"]]

# convert('ts', float, int)
life_data['ts'] = life_data['ts'].astype(int)

# convert('userId', object, int)
life_data['userId'] = life_data['userId'].str.replace('user', '').astype(int)

# merge requirement
life_data['date'] = pd.to_datetime(life_data['ts'], unit='s')
life_data['date'] = life_data['date'].dt.date
life_data['date'] = pd.to_datetime(life_data['date'])

life_data


,userId,ts,actionOption,date
0,1,1599531660,132,2020-09-08
1,1,1599531720,132,2020-09-08
2,1,1599531840,132,2020-09-08
3,1,1599531840,132,2020-09-08
4,1,1599531900,132,2020-09-08
...,...,...,...,...
817,30,1598966160,725,2020-09-01
818,30,1598966220,725,2020-09-01
819,30,1598966280,725,2020-09-01
820,30,1598966340,725,2020-09-01


In [9]:
'''
신체활동 분류표에 따른 강도 분류:
즉 신체활동강도는 METs 값에 따라 
좌식행동 (1.0–1.5 METs)은 1 
저강도 (1.6–2.9 METs)는 2
중강도 (3.0–5.9 METs)는 3
고강도 (≥ 6.0 METs) 는 4
'''

strength_0 = [111, 112]
strength_1 = [22, 61, 63, 91, 121, 122, 131, 134, 213, 312, 313, 321, 322, 712, 
              713, 721, 722, 723, 724, 725, 741, 742, 761, 792, 793]
strength_2 = [41, 42, 43, 45, 46, 51, 52, 53, 55, 57, 62, 64, 92, 711, 751, 
              762, 133, 211, 212, 311, 314, 743, 764, 763, 132]
strength_3 = [44, 54, 56, 81, 82, 83, 84, 85, 86, 87, 744, 744, 745, 746, 756, 756, 791]
strength_4 = [752, 753, 754, 755]

for i, strength in enumerate([strength_0, strength_1, strength_2, strength_3, strength_4]):
    life_data.loc[life_data["actionOption"].isin(strength), "actionOption"] = i
    
life_data

,userId,ts,actionOption,date
0,1,1599531660,2,2020-09-08
1,1,1599531720,2,2020-09-08
2,1,1599531840,2,2020-09-08
3,1,1599531840,2,2020-09-08
4,1,1599531900,2,2020-09-08
...,...,...,...,...
817,30,1598966160,1,2020-09-01
818,30,1598966220,1,2020-09-01
819,30,1598966280,1,2020-09-01
820,30,1598966340,1,2020-09-01


In [10]:
'''
Resampling
- frequency option
1H  :   H
1/2H:   30T

'''

frequency = 'H' # frequency = '30T'

life_data['ts'] = pd.to_datetime(life_data['ts'], unit='s')  # Unix timestamp를 datetime으로 변환
resampled_life_data = pd.DataFrame(index=life_data.groupby(['userId', pd.Grouper(freq=frequency, key='ts')]).first().index)

for column in life_data.columns:
    resampled_life_data[column] = life_data.groupby(['userId', pd.Grouper(freq=frequency, key='ts')])[column].apply(lambda x: x.mode()[0] if len(x) > 0 else np.nan)

# groupby index remove
resampled_life_data.drop(['ts','userId'], axis=1, inplace=True)
resampled_life_data = resampled_life_data.reset_index()

resampled_life_data

,userId,ts,actionOption,date
0,1,2020-08-30 03:00:00,2,2020-08-30
1,1,2020-08-30 04:00:00,2,2020-08-30
2,1,2020-08-30 05:00:00,1,2020-08-30
3,1,2020-08-30 06:00:00,1,2020-08-30
4,1,2020-08-30 07:00:00,1,2020-08-30
...,...,...,...,...
9146,30,2020-09-27 09:00:00,1,2020-09-27
9147,30,2020-09-27 10:00:00,1,2020-09-27
9148,30,2020-09-27 11:00:00,1,2020-09-27
9149,30,2020-09-27 12:00:00,1,2020-09-27


In [11]:
# Merging resampled_life_data, sleep_data, user_info_data

final_data = pd.merge(resampled_life_data, sleep_data, on=["userId", "date"])
final_data = pd.merge(final_data, user_info_data, on=["userId"])
# life_data # rows: 440830.
# sleep_data # rows: 615.

#date column 삭제 (ts: 유닉스에포크타임 존재)
final_data.drop(['date'], axis=1, inplace=True)

final_data['ts'] = (final_data['ts'].apply(lambda x: pd.to_datetime(x, unit='s')).apply(lambda x: x.timestamp())).astype(int)

#final_data 데이터 프레임 자료형 출력
print(final_data.dtypes)
final_data # rows: 414380

userId                int64
ts                    int64
actionOption          int64
lightsleepduration    int64
deepsleepduration     int64
wakeupcount           int64
remsleepduration      int64
sleep_score           int64
sleepTime             int64
gender                int64
age                   int64
bmi                   int64
dtype: object


,userId,ts,actionOption,lightsleepduration,deepsleepduration,wakeupcount,remsleepduration,sleep_score,sleepTime,gender,age,bmi
0,1,1598832000,2,8190,1836,1,7500,83,28740,1,27,27
1,1,1598835600,2,8190,1836,1,7500,83,28740,1,27,27
2,1,1598839200,2,8190,1836,1,7500,83,28740,1,27,27
3,1,1598842800,2,8190,1836,1,7500,83,28740,1,27,27
4,1,1598846400,1,8190,1836,1,7500,83,28740,1,27,27
...,...,...,...,...,...,...,...,...,...,...,...,...
8647,30,1601110800,1,6258,450,9,6960,32,21120,1,26,19
8648,30,1601114400,0,6258,450,9,6960,32,21120,1,26,19
8649,30,1601118000,1,6258,450,9,6960,32,21120,1,26,19
8650,30,1601121600,1,6258,450,9,6960,32,21120,1,26,19


In [12]:
#Column integration
if csv == "cnn":
    final_CNN = final_data
    cols = list(final_CNN.columns)
    cols.insert(0, cols.pop(cols.index('sleep_score')))
    final_CNN = final_CNN.loc[:, cols]


    final_CNN['feature'] = final_CNN.iloc[:, 1:].apply(lambda x: ' '.join(x.astype(str)), axis=1)
    final_CNN = final_CNN.iloc[:, [0, -1]]
    final_CNN
    
elif csv == "bert":
    final_BERT = final_data
    cols = list(final_BERT.columns)
    cols.append(cols.pop(cols.index('sleep_score')))
    final_BERT = final_BERT[cols]

    # 첫번째 열과 마지막 열 선택
    cols = ['userId'] + list(final_BERT.columns[1:-1]) + ['sleep_score']
    final_BERT = final_BERT[cols]

    # 가운데 열의 값을 공백으로 합치기
    final_BERT['feature'] = final_BERT.iloc[:, 1:-1].astype(int).apply(lambda x: ' '.join(x.astype(str)), axis=1)

    # 열 순서 정렬
    final_BERT = final_BERT[['userId', 'feature', 'sleep_score']]
    final_BERT['sleep_score'] = final_BERT['sleep_score'].astype(str)
    final_BERT
else:
    print("Model Not Found")

In [13]:
final_CNN 
# final_BERT

,sleep_score,feature
0,83,1 1598832000 2 8190 1836 1 7500 28740 1 27 27
1,83,1 1598835600 2 8190 1836 1 7500 28740 1 27 27
2,83,1 1598839200 2 8190 1836 1 7500 28740 1 27 27
3,83,1 1598842800 2 8190 1836 1 7500 28740 1 27 27
4,83,1 1598846400 1 8190 1836 1 7500 28740 1 27 27
...,...,...
8647,32,30 1601110800 1 6258 450 9 6960 21120 1 26 19
8648,32,30 1601114400 0 6258 450 9 6960 21120 1 26 19
8649,32,30 1601118000 1 6258 450 9 6960 21120 1 26 19
8650,32,30 1601121600 1 6258 450 9 6960 21120 1 26 19


In [14]:
#CSV download
if csv == "cnn":
    CNN_path = "/Users/ideal/Code/jupyter/csv_storage/lifelog.csv"
    final_CNN.to_csv(CNN_path, index=False, encoding="utf-8-sig")


elif csv == "bert":
    BERT_path = "/Users/ideal/Code/jupyter/csv_storage/BERT_total.csv"
    final_BERT.to_csv(BERT_path, index=False, encoding="utf-8-sig")

In [15]:
#Separating BERT train, test csv
if csv == "bert":
    # Read CSV file
    df = pd.read_csv(BERT_path, index_col=False)

    # Set the ratio of the test data
    test_ratio = 0.3

    # Split data into train and test sets
    n_test = int(test_ratio * df.shape[0])
    df_test = df.sample(n=n_test, random_state=42)
    df_train = df.drop(df_test.index)

    # df_train.drop(['Unnamed: 0'], axis=1, inplace=True)
    # df_test.drop(['Unnamed: 0'], axis=1, inplace=True)

    # Save data to separate files
    train_path = os.path.join(os.path.dirname(BERT_path), "BERT_train.csv")
    test_path = os.path.join(os.path.dirname(BERT_path), "BERT_test.csv")
    df_train.to_csv(train_path, index=False)
    df_test.to_csv(test_path, index=False)

    print("Finish")
